# Cifar10 model testing
In this notebook we will try a different approach for the context learning methodology. Instead of using a convolutional architecture, we will use a recurrent neural network.

## Make the necessary imports

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
!pip install umap-learn[plot]

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from classification_models.tfkeras import Classifiers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import math
import shutil
import os
import gc

import umap.umap_ as umap
import umap.plot

## Import and preprocess the Cifar10 dataset

In [ ]:
(Xtrain_orig, ytrain_orig), (Xtest_orig, ytest_orig) = cifar10.load_data()

#We do not normalize the data as it it part of the training process with EfficientNet
Xtrain = Xtrain_orig.astype('float32')
Xtest = Xtest_orig.astype('float32')

ytrain = ytrain_orig
ytest = ytest_orig

print ("Xtrain shape: " + str(Xtrain.shape))
print ("ytrain shape: " + str(ytrain.shape))

print ("Xtest shape: " + str(Xtest.shape))
print ("ytest shape: " + str(ytest.shape))


# Support functions

In [ ]:
def create_random_set(X, y, strip_size=4, set_size=60000):
    
    #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We want the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 3])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, 0] = tf.expand_dims(tf.squeeze(X[X_groups[i][j]]), axis=0)
            X_processed[i, j:j+1, :, :, 1] = tf.expand_dims(tf.squeeze(X[X_groups[i][j]]), axis=0)
            X_processed[i, j:j+1, :, :, 2] = tf.expand_dims(tf.squeeze(X[X_groups[i][j]]), axis=0)
        X_processed[i, strip_size, :, :, 0] = tf.expand_dims(tf.squeeze(X[X_groups[i][strip_size]]), axis=0)
        X_processed[i, strip_size, :, :, 1] = tf.expand_dims(tf.squeeze(X[X_groups[i][strip_size]]), axis=0)
        X_processed[i, strip_size, :, :, 2] = tf.expand_dims(tf.squeeze(X[X_groups[i][strip_size]]), axis=0)
        y_processed[i] = y_label[i] 
    return X_processed, y_processed

In [ ]:
def create_random_set_RGB(X, y, strip_size=4, set_size=60000):
    
    #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We want the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 3])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, :] = X[X_groups[i][j]]
        X_processed[i, strip_size, :, :, :] = X[X_groups[i][strip_size]]
        y_processed[i] = y_label[i]
        
        
    return X_processed, y_processed

In [ ]:
def create_random_data_sets(X, y, Xt, yt, strip_size, RGB, training_size=30000, test_size=8000):
    if(RGB):
        Xtrain, ytrain = create_random_set_RGB(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.1)
        Xtest, ytest = create_random_set_RGB(Xt, yt, strip_size, set_size=test_size)
    else:
        #Transform the images to black and white
        Xtrain, ytrain = create_random_set(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.1)
        Xtest, ytest = create_random_set(Xt, yt, strip_size, set_size=test_size)
    
    print ("Training examples classified as 0: " + str(len(np.where(ytrain==0)[0])))
    print ("Training examples classified as 1: " + str(len(np.where(ytrain==1)[0])))
    print ("Validation examples classified as 0: " + str(len(np.where(yval==0)[0])))
    print ("Validation examples classified as 1: " + str(len(np.where(yval==1)[0])))
    print ("Test examples classified as 0: " + str(len(np.where(ytest==0)[0])))
    print ("Test examples classified as 1: " + str(len(np.where(ytest==1)[0])))
    
    return Xtrain, Xval, Xtest, ytrain, yval, ytest

In [ ]:
def train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, batch_size=32, model_save_name="best_model"):

    #Define callbacks
    #Save the best model
    dirname = os.getcwd()
    filepath = os.path.join(dirname, model_save_name)
    filepath = os.path.join(filepath, 'model')
    
    model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
        mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
    #Add early stopping
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
    #Reduce learning rate on plateau
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.65, patience=5, min_lr=0.00001, verbose=1)

    callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

    #Compile and fit the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(Xtrain, ytrain,
                        batch_size=batch_size,
                        epochs=100,
                        validation_data=(Xval, yval),
                        callbacks=callbacks,
                        verbose=1)
    
    """plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper right')


    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper right')"""
    
    model.load_weights(filepath)
    ypredict = model.predict(Xtest)
    #ypredict = tf.squeeze(ypredict).numpy()
    #print(ypredict)
    #ypredict_round = [round(x) for x in ypredict]
    score = model.evaluate(Xtest, ytest, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    
    #cm = confusion_matrix(ytest, ypredict)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    #disp.plot()
    #plt.show()
    
    return score[1], model

In [ ]:
def see_embeddings(model, X, y, save_name, preprocess_func, pre_model=None):
    embeddings_per_class = 10
    classes = 10
    # Buscamos 3 imagenes de cada clase
    #Creamos una lista de listas donde cada sublista contiene los índices de la imágenes de un número
    """list_indices_by_number = [np.where(y == i)[0] for i in range(classes)]
    
    images = np.zeros((classes*embeddings_per_class, X.shape[1], X.shape[2], X.shape[3]))
    for i in range(classes):
        list_of_indexes = random.choices(list_indices_by_number[i], k=embeddings_per_class)
        images[embeddings_per_class*i:(i+1)*embeddings_per_class, :, :, :] = X[list_of_indexes]
"""
    if preprocess_func:
        #prep_images = preprocess_func(images)
        prep_X = preprocess_func(X)
    else:
        #prep_images = images
        prep_X = X

    if pre_model:
        #prep_images = model.preprocess(prep_images)
        prep_X = model.preprocess(prep_X)
    
    if prep_X.shape[-1] != 3:
        #images_3d = np.zeros((classes*embeddings_per_class, prep_images.shape[1], prep_images.shape[2], 3))
        #for i in range(classes*embeddings_per_class):
            #images_3d[i, :, : , :] = np.repeat(prep_images[i, :, :, :], 3, 2)
        #prep_images = images_3d
        prep_X = tf.repeat(prep_X, 3, axis=-1)

    #encoded_images = model.encoder(prep_images).numpy()
    encoded_X = model.encoder(prep_X).numpy()
    
    """plt.figure(figsize=(15, 30))

    m = embeddings_per_class
    n = classes
    for i in range(m):
        for j in range(n):
            ax = plt.subplot(m, n, n*i+j+1)
            plt.imshow(tf.expand_dims(encoded_images[n*i+j], axis=-1))
            plt.title('Class: ' + str(i), fontsize=16)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.suptitle('Image embeddings per class', fontsize=20)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.savefig(save_name+ '.png')
    plt.show()"""
    
    manifold = umap.UMAP().fit(encoded_X)
    p = umap.plot.points(manifold, labels=tf.squeeze(y), color_key_cmap='Paired', background='black')
    plt.savefig(save_name+'_UMAP'+ '.png')
    umap.plot.plt.show()

In [ ]:
def save_to_db(db_name, column_name, list_to_save):
    df = pd.read_csv(db_name + '.csv')
    df[column_name] = list_to_save
    df.to_csv(db_name + '.csv', index=False)

def create_db(db_name):
    df = pd.DataFrame()
    df.to_csv(db_name + '.csv', index=True)

def plot_db_columns(db_name, title, xlabel, ylabel, save_name):
    df = pd.read_csv(db_name + '.csv')
    # plot lines
    x = [i for i in range(1, 10)]
    plt.figure(figsize=(9,7))
    
    for column in df:
        if (column != 'Unnamed: 0'):
            plt.plot(x, df[column], label = column)

    plt.title(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.5)
    plt.tick_params(labelsize=14)
    plt.savefig(save_name + '.png')
    plt.show()

In [ ]:
def run_model(X, y, Xt, yt, model_class, latent_dim, model_save_name, db_name, column_name, preprocess_func, preprocess_before, num_iterations=3, RGB=True, pre_model=None):
    
    #Preprocess data
    if preprocess_before:
        X_processed = preprocess_func(X)
        Xt_processed = preprocess_func(Xt)
    else:
        X_processed = X
        Xt_processed = Xt
        
    accuracy_per_strip_size = []
    for strip_size in range(1, 10):
        print('-------------------' + str(strip_size) + '-------------------')
        channels = strip_size + 1
        accuracy_per_iteration = []
        for i in range(num_iterations):
            print('--------------Iteration ' + str(i+1) + '--------------')
            Xtrain, Xval, Xtest, ytrain, yval, ytest = create_random_data_sets(X_processed, y, Xt_processed, yt, strip_size, RGB)
            tf.keras.backend.clear_session()
            if pre_model:
                model = model_class(latent_dim, channels, (None, Xtrain[0].shape[1], Xtrain[0].shape[2], Xtrain[0].shape[3]), pre_model)
            else:
                model = model_class(latent_dim, channels, (None, Xtrain[0].shape[1], Xtrain[0].shape[2], Xtrain[0].shape[3])) 
            score, model = train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, batch_size=64, model_save_name=model_save_name)
            if i == (num_iterations-1):
                embedding_file_path = os.path.join('embedding_images', model_save_name + "_" + str(channels))
                see_embeddings(model, Xt, yt, embedding_file_path, preprocess_func)
            accuracy_per_iteration.append(score)
            del Xtrain
            del Xval
            del Xtest
            del ytrain
            del yval
            del ytest
            gc.collect()
        mean_score = np.mean(accuracy_per_iteration)
        accuracy_per_strip_size.append(mean_score)
    save_to_db(db_name, column_name, accuracy_per_strip_size)

## Define the models

## Functions to build the models

In [ ]:
def build_efficientnet_encoder(shape_in, latent_dim):
  
    inp = layers.Input(shape=shape_in)
 
    # Apply EfficientNet to every image
    efficientnet = keras.applications.EfficientNetB0(include_top=False, pooling='avg', weights='imagenet', drop_connect_rate=.5)
    x = efficientnet(inp)
    
    x = layers.Dropout(.8)(x)
    x = layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    x = layers.Dropout(.5)(x)
    # Embedding de cada imatge (dim reduced from 2048 to encoding_dim)
    x = layers.Dense(latent_dim, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    encoder = keras.Model(inputs=inp, outputs=x)

    return encoder

In [ ]:
def build_pretrained_efficientnet_encoder(shape_in, latent_dim):

    inp = layers.Input(shape=shape_in)
 
    # Apply EfficientNet to every image
    efficientnet = keras.applications.EfficientNetB0(include_top=False, pooling='avg', weights='imagenet', drop_connect_rate=.5)
    x = efficientnet(inp)
    
    x = layers.Dropout(.8)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(.5)(x)
    # Embedding de cada imatge (dim reduced from 2048 to encoding_dim)
    x = layers.Dense(latent_dim, activation='relu')(x)
    encoder = keras.Model(inputs=inp, outputs=x)

    dirname = os.getcwd()
    filepath = os.path.join(dirname, 'pretrained_efficientnet' + str(latent_dim))
    filepath = os.path.join(filepath, 'model')
    encoder.load_weights(filepath)

    return encoder

In [ ]:
def build_resnet_encoder(shape_in, latent_dim):

    inp = layers.Input(shape=shape_in)
 
    # Apply EfficientNet to every image
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    resnet = ResNet18(shape_in, weights='imagenet', include_top=False, pooling='avg')
    x = resnet(inp)
    
    x = layers.Flatten()(x)
    x = layers.Dropout(.8)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    x = layers.Dropout(.5)(x)
    # Embedding de cada imatge (dim reduced from 2048 to encoding_dim)
    x = layers.Dense(latent_dim, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    encoder = keras.Model(inputs=inp, outputs=x)

    return encoder

In [ ]:
def build_pretrained_resnet_encoder(shape_in, latent_dim):
  
    inp = layers.Input(shape=shape_in)
 
    # Apply EfficientNet to every image
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    resnet = ResNet18(shape_in, weights='imagenet', include_top=False, pooling='avg')
    x = resnet(inp)
    
    x = layers.Flatten()(x)
    x = layers.Dropout(.8)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    x = layers.Dropout(.5)(x)
    # Embedding de cada imatge (dim reduced from 2048 to encoding_dim)
    x = layers.Dense(latent_dim, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    encoder = keras.Model(inputs=inp, outputs=x)

    dirname = os.getcwd()
    filepath = os.path.join(dirname, 'pretrained_resnet' + str(latent_dim))
    filepath = os.path.join(filepath, 'model')
    encoder.load_weights(filepath)

    return encoder

In [ ]:
def build_lstm_model(shape_in, encoder):
    inp = layers.Input(shape=shape_in)

    encoded_out = layers.TimeDistributed(encoder)(inp)

    gru1 = layers.GRU(256, return_sequences=True, recurrent_dropout=.7)(encoded_out)
    gru2 = layers.GRU(128, return_sequences=True, recurrent_dropout=.7)(gru1)
    gru3 = layers.GRU(64, return_sequences=False, recurrent_dropout=.7)(gru2)

    dense3 = layers.Dense(128, activation='relu')(gru3)
    dense4 = layers.Dense(64, activation='relu')(dense3)
    drop3 = layers.Dropout(.5)(dense4)

    dense4 = layers.Dense(64, activation='relu')(drop3)
    out = layers.Dense(1, activation='sigmoid')(dense4)

    model = keras.Model(inputs=inp, outputs=out)

    return model

## Build the models

In [ ]:
class EfficientNetModel(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(EfficientNetModel, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = build_pretrained_efficientnet_encoder(self.shape_in[1:], self.latent_dim)

        self.model = build_lstm_model(self.shape_in, self.encoder)

    def call(self, x):

        y_predict = self.model(x)
        
        return y_predict

In [ ]:
class ResNetModel(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ResNetModel, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = build_pretrained_resnet_encoder(self.shape_in[1:], self.latent_dim)

        self.model = build_lstm_model(self.shape_in, self.encoder)

    def call(self, x):

        y_predict = self.model(x)
        
        return y_predict

## First train EfficientNetB0 as classifier
We want to first train the EfficientNetB0 as a classifier so we can then load the weights to the encoder in the model.

In [ ]:

def preprocess_data(X, Y):
    X_p = keras.applications.efficientnet.preprocess_input(X)
    Y_p = keras.utils.to_categorical(Y, 32)
    return X, Y_p

efficientnet_model = build_efficientnet_encoder((32, 32, 3), 32)

X, y = preprocess_data(Xtrain, ytrain)
X, Xv, y, yv = train_test_split(X, y, test_size=0.2)

datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(X)

#Define callbacks
#Save the best model
dirname = os.getcwd()
filepath = os.path.join(dirname, 'pretrained_efficientnet32')
filepath = os.path.join(filepath, 'model')

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
    mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
#Add early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
#Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.65, patience=5, min_lr=0.000001, verbose=1)

callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

#Compile and fit the model
efficientnet_model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history = efficientnet_model.fit_generator(datagen.flow(X, y, batch_size=128), steps_per_epoch = len(X) / 128, epochs=200, validation_data=(Xv, yv), callbacks=callbacks)

plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper right')


plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper right')

efficientnet_model.load_weights(filepath)

Xt, yt = preprocess_data(Xtest, ytest)

ypredict = efficientnet_model.predict(Xt)

#ypredict = tf.squeeze(ypredict).numpy()
#print(ypredict)
#ypredict_round = [round(x) for x in ypredict]
score = efficientnet_model.evaluate(Xt, yt, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:

def preprocess_data(X, Y):
    X_p = keras.applications.efficientnet.preprocess_input(X)
    Y_p = keras.utils.to_categorical(Y, 10)
    return X, Y_p

efficientnet_model = build_efficientnet_encoder((32, 32, 3), 10)

X, y = preprocess_data(Xtrain, ytrain)
X, Xv, y, yv = train_test_split(X, y, test_size=0.2)

datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(X)

#Define callbacks
#Save the best model
dirname = os.getcwd()
filepath = os.path.join(dirname, 'pretrained_efficientnet10')
filepath = os.path.join(filepath, 'model')

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
    mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
#Add early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
#Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.65, patience=5, min_lr=0.000001, verbose=1)

callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

#Compile and fit the model
efficientnet_model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history = efficientnet_model.fit_generator(datagen.flow(X, y, batch_size=128), steps_per_epoch = len(X) / 128, epochs=200, validation_data=(Xv, yv), callbacks=callbacks)

plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper right')


plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper right')

efficientnet_model.load_weights(filepath)

Xt, yt = preprocess_data(Xtest, ytest)

ypredict = efficientnet_model.predict(Xt)

#ypredict = tf.squeeze(ypredict).numpy()
#print(ypredict)
#ypredict_round = [round(x) for x in ypredict]
score = efficientnet_model.evaluate(Xt, yt, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## First train Resnet18 as classifier
We want to first train the Resnet18 as a classifier so we can then load the weights to the encoder in the model.

In [ ]:
"""#Creamos el modelo
def preprocess_data(X, Y):
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    X_p = preprocess_input(X)
    Y_p = keras.utils.to_categorical(Y, 32)
    return X, Y_p

resnet_model = build_resnet_encoder((32, 32, 3), 32)

X, y = preprocess_data(Xtrain, ytrain)
X, Xv, y, yv = train_test_split(X, y, test_size=0.2)

datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(X)

#Define callbacks
#Save the best model
dirname = os.getcwd()
filepath = os.path.join(dirname, 'pretrained_resnet32')
filepath = os.path.join(filepath, 'model')

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
    mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
#Add early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
#Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.65, patience=5, min_lr=0.000001, verbose=1)

callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

#Compile and fit the model
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history = resnet_model.fit_generator(datagen.flow(X, y, batch_size=128), steps_per_epoch = len(X) / 128, epochs=200, validation_data=(Xv, yv), callbacks=callbacks)


plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper right')


plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper right')

resnet_model.load_weights(filepath)

Xt, yt = preprocess_data(Xtest, ytest)

ypredict = resnet_model.predict(Xt)
#ypredict = tf.squeeze(ypredict).numpy()
#print(ypredict)
#ypredict_round = [round(x) for x in ypredict]
score = resnet_model.evaluate(Xt, yt, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])"""

In [ ]:
"""#Creamos el modelo
def preprocess_data(X, Y):
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    X_p = preprocess_input(X)
    Y_p = keras.utils.to_categorical(Y, 10)
    return X, Y_p

resnet_model = build_resnet_encoder((32, 32, 3), 10)

X, y = preprocess_data(Xtrain, ytrain)
X, Xv, y, yv = train_test_split(X, y, test_size=0.2)

datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )
datagen.fit(X)

#Define callbacks
#Save the best model
dirname = os.getcwd()
filepath = os.path.join(dirname, 'pretrained_resnet10')
filepath = os.path.join(filepath, 'model')

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
    mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
#Add early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose = 0)
#Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.65, patience=5, min_lr=0.000001, verbose=1)

callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

#Compile and fit the model
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history = resnet_model.fit_generator(datagen.flow(X, y, batch_size=128), steps_per_epoch = len(X) / 128, epochs=200, validation_data=(Xv, yv), callbacks=callbacks)


plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper right')


plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper right')

resnet_model.load_weights(filepath)

Xt, yt = preprocess_data(Xtest, ytest)

ypredict = resnet_model.predict(Xt)
#ypredict = tf.squeeze(ypredict).numpy()
#print(ypredict)
#ypredict_round = [round(x) for x in ypredict]
score = resnet_model.evaluate(Xt, yt, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])"""

# Entrenamos los modelos

In [ ]:
os.mkdir('embedding_images')
create_db('cifar10_rnn')

In [ ]:
#Modelos con conv encoder para Cifar10
run_model(Xtrain, ytrain, Xtest, ytest, EfficientNetModel, 32, 'embedding_efficient32', 'cifar10_rnn', 'embedding_efficient32', keras.applications.efficientnet.preprocess_input, True, num_iterations=1, RGB=True)
run_model(Xtrain, ytrain, Xtest, ytest, EfficientNetModel, 10, 'embedding_efficient10', 'cifar10_rnn', 'embedding_efficient10', keras.applications.efficientnet.preprocess_input, True, num_iterations=1, RGB=True)


In [ ]:
"""ResNet18, preprocess_input = Classifiers.get('resnet18')
run_model(Xtrain, ytrain, Xtest, ytest, ResNetModel, 32, 'embedding_resnet', 'cifar10_rnn', 'embedding_resnet32', preprocess_input, True, num_iterations=1, RGB=True)
run_model(Xtrain, ytrain, Xtest, ytest, ResNetModel, 10, 'embedding_resnet', 'cifar10_rnn', 'embedding_resnet10', preprocess_input, True, num_iterations=1, RGB=True)"""

# Ploteamos los resultados

In [ ]:
plot_db_columns('cifar10_rnn', 'Cifar10 model accuracy', 'Strip size', 'Accuracy', 'cifar10_rnn')